In [ ]:
####################
#June 25th, 2025
#Joining the trans and cis into a single file, but instead of splitting by chromosome keeping them all together, merge all 3 replicates
#merging the pairs file didn't work as I have to sort them later and reheader them and that's more complicated
#Generating a filtered pairs file for each replicate, transforming them to coolers and then merging the coolers

import sys
import gzip

def filter_pairs_by_specific_chromosomes(input_file, output_file):
    # List of chromosomes to filter
    target_chromosomes = ["2L_JJg14_439","2R_JJg14_439", "3L_JJg14_439", "3R_JJg14_439", "4_JJg14_439", "X_JJg14_439", "Y_JJg14_439", "2L_JJg14_057", "2R_JJg14_057", "3L_JJg14_057", "3R_JJg14_057", "4_JJg14_057", "X_JJg14_057", "Y_JJg14_057"]
    buffer = []
    
    buffer_size = 10000

    with gzip.open(input_file, 'rt') as infile, gzip.open(output_file, 'wt') as outfile:
        for line in infile:
            if line.startswith("#"):  # Process header lines
                # Check if the header line starts with any of the target chromosomes
                if line.startswith("#chromsize"):
                    parts = line.strip().split(" ")
                    if parts[1] in target_chromosomes:
                        #create a new chromosome trans and add it to the header
                        line = line.replace("_JJg14_057", "_trans")
                        line = line.replace("_JJg14_439", "_trans")
                        buffer.append(line)
                        #and then replace trans with nothing and add the sis chromosomes to header
                        line = line.replace("_trans", "")
                        buffer.append(line)
                elif line.startswith("#samheader: @SQ"):
                    parts = line.strip().split(":")
                    parts2 = parts[2].strip().split("LN")
                    if parts2[0].strip() in target_chromosomes:
                        line = line.replace("_JJg14_057", "_trans")
                        line = line.replace("_JJg14_439", "_trans")
                        buffer.append(line)
                        line = line.replace("_trans", "")
                        buffer.append(line)
                else:
                    buffer.append(line)

                outfile.writelines(buffer)
                buffer=[]
                continue
            parts = line.strip().split("\t")
            # Check if both parts[1] and parts[3] are in the target chromosomes list
            if parts[1] in target_chromosomes and parts[3] in target_chromosomes:
                
                if parts[1] == parts[3]:
                    line = line.replace("_JJg14_057", "")
                    line = line.replace("_JJg14_439", "")
                else:
                    line = line.replace("_JJg14_057", "_trans")
                    line = line.replace("_JJg14_439", "_trans")
                
                buffer.append(line)
      
                if len(buffer) >= buffer_size:
                    outfile.writelines(buffer)
                    buffer=[]

        
        
input_file1 = "/data/alhajabed/FlyHiC/PoreC/20250625_balancedMcools/24L004895_PnM1.pairs.gz"
input_file2 = "/data/alhajabed/FlyHiC/PoreC/20250625_balancedMcools/24L004896_PnM2.pairs.gz"
input_file3 = "/data/alhajabed/FlyHiC/PoreC/20250625_balancedMcools/24L004897_PnM3.pairs.gz"
output_file1 = "/data/alhajabed/FlyHiC/PoreC/20250625_balancedMcools/24L004895_PnM1.filt.pairs.gz"
output_file2 = "/data/alhajabed/FlyHiC/PoreC/20250625_balancedMcools/24L004896_PnM2.filt.pairs.gz"
output_file3 = "/data/alhajabed/FlyHiC/PoreC/20250625_balancedMcools/24L004897_PnM3.filt.pairs.gz"
  
filter_pairs_by_specific_chromosomes(input_file1, output_file1)
filter_pairs_by_specific_chromosomes(input_file2, output_file2)
filter_pairs_by_specific_chromosomes(input_file3, output_file3)


In [ ]:
#pairs to cool to mcool

import subprocess
import cooler
import os


pairs_file = "/data/alhajabed/FlyHiC/PoreC/20250625_balancedMcools/20250625_mergedPnMs.pairs.gz"
sorted_pairs = "/data/alhajabed/FlyHiC/PoreC/20250625_balancedMcools/20250625_mergedPnMs.sorted.pairs.gz"
chrom_sizes = "dm6.chrom.sizes"  # Replace with your Drosophila chrom sizes file
bin_size = 10000
bins_bed = f"bins_{bin_size}.bed"
cool_file = f"output_{bin_size}.cool"
mcool_file = f"{cool_file}::resolutions/{bin_size}"

# --- Step 1: Sort the pairs file ---
print("Sorting .pairs file...")
subprocess.run(f"zcat {pairs_file} | sort -k2,2 -k3,3n -k4,4 -k5,5n | bgzip  > {sorted_pairs}", shell=True, check=True)

# --- Step 2: Generate bins file ---
print("Generating bins...")
subprocess.run(f"cooler makebins {chrom_sizes} {bin_size} > {bins_bed}", shell=True, check=True)

# --- Step 3: Load into .cool format ---
print("Loading into .cool file...")
subprocess.run(f"cooler cload pairs --assembly dm6 {bins_bed} {sorted_pairs} {cool_file}", shell=True, check=True)

# --- Step 4: Zoomify to create .mcool ---
print("Zoomifying into .mcool...")
subprocess.run(f"cooler zoomify {cool_file}", shell=True, check=True)

print(f"✅ Done! Your multi-resolution file is: {cool_file.replace('.cool', '.mcool')}")